In [179]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [180]:
from bs4 import BeautifulSoup

# Load html file and filter to only <p> tags
with open('/Users/vinayakkannan/Desktop/Projects/FactChecker/Scraping/Raw Data/climate_change_epa.html', 'r') as file:
    soup = BeautifulSoup(file, 'html')
    paragraphs = soup.find_all('p')

In [181]:
# Get only the text from the <p> tags
text = [p.get_text() for p in paragraphs]
# Drop last 9 elements as they are not relevant
text = text[:-9]
print(len(text))

601


In [182]:
# Get text from <li> tags
lis = soup.find_all('li')
li_text = [li.get_text() for li in lis]
print(li_text)
# append to text
text.extend(li_text)

['Engage. Include people with diverse backgrounds and experiences in community efforts to address climate change. Foster honest conversation, meet people where they are already active (such as in schools and community centers), and involve those leaders who are respected by the community. Indigenous and local knowledge can advance understanding of climate change effects and solutions. When possible, amplify the voices of those who are most vulnerable, underserved, or overburdened.\xa0', 'Share. Connect communities with the information and data that can help them prepare for and adapt to climate impacts. Helping communities access relevant resources enables them to refine their climate resiliency efforts and empowers them to develop targeted, grassroots solutions.', 'Build. Climate-resilient solutions based on equity and inclusivity reduce vulnerability for everyone. Consider solutions that boost resilience while improving livelihoods, accessibility, and social and economic well-being. 

In [183]:
# Use nltk to split the text into sentences
# Put sentences into a dataframe
import pandas as pd
df = pd.DataFrame(text, columns=['claims'])
# Add column for veracity where all the entries are 3
df['veracity'] = 3

In [184]:
from Scraping.Helpers.RedditScraper import RedditScraper

scraper = RedditScraper(limit=10000, claim_limit=10000)
output = scraper.get_claims(df, 'claims')

2621


1it [00:00,  3.08it/s]

{'version': '2', 'sentences': 'Climate change does not affect all people equally. ', 'results': [{'text': 'Climate change does not affect all people equally.', 'index': 0, 'score': 0.5364307229}]}


2621it [00:21, 121.71it/s]

{'version': '2', 'sentences': 'Some communities experience disproportionate impacts because of existing vulnerabilities, historical patterns of inequity, socioeconomic disparities, and systemic environmental injustices (e.g., redlining). People who already face the greatest burdens are often the ones affected most by climate change.. EPA is committed to supporting communities—particularly those facing disproportionate impacts—develop and implement equitable solutions to climate change impacts. Only through collaboration across all communities and levels of government can the nation make progress in addressing systemic factors that impact climate equity.. Climate equity is the goal of recognizing and addressing the unequal burdens made worse by climate change, while ensuring that all people share the benefits of climate protection efforts. Achieving equity means that all people—regardless of their race, color, gender, age, sexuality, national origin, ability, or income—live in safe, hea

In [185]:
output_preserved = output

In [186]:
output_preserved

,text,score,index
0,Climate change does not affect all people equa...,0.536431,0
1,Some communities experience disproportionate i...,0.593216,0
2,People who already face the greatest burdens a...,0.560092,1
3,Only through collaboration across all communit...,0.226200,2
4,Climate equity is the goal of recognizing and ...,0.473207,3
...,...,...,...
2676,See answers to common questions on climate cha...,0.613346,2675
2677,Learn more about Climate Change Indicators.. C...,0.526464,2676
2678,2.,0.404242,2677
2679,"Increases in ocean temperatures, sea level, an...",0.626390,2678


In [220]:
# Reset index on output_preserved
output = output_preserved

In [221]:
import re

# Drop index column from output
output = output.drop(columns=['index'])
# Add veracity column to output, where all the entries are 3
output['Numerical Rating'] = 3
# Filter output to where score is greater than 0.7
output = output[output['score'] >= 0.7]

# Rename 'text' column to 'Text'
output = output.rename(columns={'text': 'Text'})

# Sort output by score
output = output.sort_values(by='score', ascending=False)

# Drop score column
output = output.drop(columns=['score'])

# Reset index
output = output.reset_index(drop=True)

# Loop through 'Text' column in output
for i in range(len(output)):
    text = output.loc[i, 'Text']
    # Find any instances where there's a '.' followed by 2 numbers and replace with '.'
    output.loc[i, 'Text'] = re.sub(r'\.\d{2}', '.', text)
    output.loc[i, 'Text'] = re.sub(r'\.\d{1}', '.', text)
    output.loc[i, 'Text'] = re.sub(r'\.\.', '.', text)

    
# Drop duplicates
output = output.drop_duplicates(subset='Text')

# Save output to csv
output.to_csv('/Users/vinayakkannan/Desktop/Projects/FactChecker/Scraping/Transformed Data/climate_change_epa_who.csv', index=False)

In [222]:
output

,Text,Numerical Rating
0,"In vulnerable regions, the death rate from ext...",3
1,"For instance, as of 2018, less than half of co...",3
2,The marine economy generated over $665 billion...,3
3,"In 2020, wildfires burned over 10 million acre...",3
5,There are over two million farms in the United...,3
...,...,...
264,Increases in average and extreme temperatures ...,3
265,About 80% of the ground in Alaska has permafro...,3
267,When a tree came in contact with electrical di...,3
270,"They also breathe at a faster rate, increasing...",3
